<a href="https://colab.research.google.com/github/npr99/PlanningMethods/blob/master/PLAN604_Comparison_of_two_proportions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Application of Comparing two Proportions from ACS Data
 
---
This Google Colab Notebook provides a complete workflow (sequence of steps from start to finish) that will allow you to explore two proportions found in ACS Data. 

This notebook compares population proportions found in the 2012 5-year ACS and the 2019 5-year ACS. The research question what census geographies have a stastically significant difference between two proportions found in two different ACS surveys. The 2012 5-year ACS includes random samples of households between 2008, 2009, 2010, 2011, and 2012. The 2019 5-year ACS includes random samples of households between 2015, 2016, 2017, 2018, and 2019.

### Two New python features in this notebook:

#### 1. Resuing code
This notebook introduces Python concepts of reading in and running a python file that includes reusable code. The code in the python file is stored on GitHub and has code that was introduced in earlier notebooks.

When a new pacakge is imported - such as `import pandas as pd` - the notebook is reading in python files that have been shared as open source. The python programming enviroment is simply many python files (`.py`) being referenced. Each (`.py`) is created and maintained by users across the globe.

#### 2. Loops
Loops are a powerful way to repeat functions across different values. In this notebook we will loop over multiple ACS years and multiple variables.


In [ ]:
# Python packages required to read in and Census API data
import requests ## Required for the Census API
import pandas as pd # For reading, writing and wrangling data
import matplotlib.pyplot as plt # For plotting and making graphs

In [ ]:
# Downloading and running python script from github
# https://jckantor.github.io/cbe61622/A.02-Downloading_Python_source_files_from_github.html
# Make sure the url is the raw version of the file on GitHub

user = "npr99"
repo = "PlanningMethods"
pyfile = "_planning_methods.py"
url = f"https://raw.githubusercontent.com/{user}/{repo}/master/{pyfile}"
!wget --no-cache --quiet --backups=1 {url}
print("Reading in python file from",url)
exec(open(pyfile).read())

Reading in python file from https://raw.githubusercontent.com/npr99/PlanningMethods/master/_planning_methods.py


## Step 1: Obtain Data
The previous step read in a python file with the Obtain Data function presented in the notebook on [Sample Size and Confidence Intervals](https://github.com/npr99/PlanningMethods/blob/master/PLAN604_Population_vs_Sample_USCounties.ipynb) The block of code with the python functions needs to be run first and then the function can be called in future blocks of code.

## Run Obtain Census API for 2012 and 2019 5-year ACS
The next block of code calls the function and gets varaibles related to public transportation use. The code is called in a loop.

For more variables see:

https://www.census.gov/data/developers/data-sets/acs-5year.2012.html

https://api.census.gov/data/2012/acs/acs5/subject/groups/S0801.html


In [ ]:
  def obtain_census_api(
                      state: str = "*",
                      county: str = "*",
                      census_geography: str = 'county:*',
                      vintage: str = "2010", 
                      dataset_name: str = 'dec/sf1',
                      get_vars: str = 'GEO_ID'):

          """General utility for obtaining census from Census API.

          Args:
              state (str): 2-digit FIPS code. Default * for all states
              county (str): 3-digit FIPS code. Default * all counties
              census_geography (str): example '&for=block:*' would be for all blocks
                default is for all counties
              vintage (str): Census Year. Default 2010
              dataset_name (str): Census dataset name. Default Decennial SF1
              for a list of all Census API
              get_vars (str): list of variables to get from the API.

          Returns:
              obj, dict: A dataframe for with Census data

          """
          # Set up hyperlink for Census API
          api_hyperlink = ('https://api.census.gov/data/' + vintage + '/'+dataset_name + '?get=' + get_vars +
                          '&in=state:' + state + '&in=county:' + county + '&for=' + census_geography)

          print("Census API data from: " + api_hyperlink)

          # Obtain Census API JSON Data
          apijson = requests.get(api_hyperlink)

          # Convert the requested json into pandas dataframe
          df = pd.DataFrame(columns=apijson.json()[0], data=apijson.json()[1:])

          return df
help(obtain_census_api)

Help on function obtain_census_api in module __main__:

obtain_census_api(state: str = '*', county: str = '*', census_geography: str = 'county:*', vintage: str = '2010', dataset_name: str = 'dec/sf1', get_vars: str = 'GEO_ID')
    General utility for obtaining census from Census API.
    
    Args:
        state (str): 2-digit FIPS code. Default * for all states
        county (str): 3-digit FIPS code. Default * all counties
        census_geography (str): example '&for=block:*' would be for all blocks
          default is for all counties
        vintage (str): Census Year. Default 2010
        dataset_name (str): Census dataset name. Default Decennial SF1
        for a list of all Census API
        get_vars (str): list of variables to get from the API.
    
    Returns:
        obj, dict: A dataframe for with Census data



In [ ]:
# Loop example
for year in ['2012','2013','2014','2015']:
  print(year)

2012
2013
2014
2015


In [ ]:
# Create an empty "container" to store multiple ACS years for the data
acs_df = {} 

dataset_name = 'acs/acs5/subject'
vintages = ['2012','2019']
get_vars = 'GEO_ID,NAME,S0801_C01_001E,S0801_C01_001M,S0801_C01_009E,S0801_C01_009M'
for vintage in vintages:
  acs_df[vintage] = obtain_census_api(get_vars = get_vars, dataset_name = dataset_name, vintage = vintage)

Census API data from: https://api.census.gov/data/2012/acs/acs5/subject?get=GEO_ID,NAME,S0801_C01_001E,S0801_C01_001M,S0801_C01_009E,S0801_C01_009M&in=state:*&in=county:*&for=county:*
Census API data from: https://api.census.gov/data/2019/acs/acs5/subject?get=GEO_ID,NAME,S0801_C01_001E,S0801_C01_001M,S0801_C01_009E,S0801_C01_009M&in=state:*&in=county:*&for=county:*


In [ ]:
acs_df['2012'].head()

,GEO_ID,NAME,S0801_C01_001E,S0801_C01_001M,S0801_C01_009E,S0801_C01_009M,state,county
0,0500000US29115,"Linn County, Missouri",5052,234,0.0,0.1,29,115
1,0500000US29091,"Howell County, Missouri",15623,560,0.3,0.3,29,091
2,0500000US29101,"Johnson County, Missouri",25227,677,0.3,0.2,29,101
3,0500000US29105,"Laclede County, Missouri",15019,481,0.1,0.2,29,105
4,0500000US29125,"Maries County, Missouri",3830,246,0.4,0.6,29,125


In [ ]:
acs_df['2019'].head()

,GEO_ID,NAME,S0801_C01_001E,S0801_C01_001M,S0801_C01_009E,S0801_C01_009M,state,county
0,0500000US17051,"Fayette County, Illinois",8633,353,0.8,0.4,17,051
1,0500000US17107,"Logan County, Illinois",12211,378,0.4,0.3,17,107
2,0500000US17165,"Saline County, Illinois",9631,334,0.9,0.6,17,165
3,0500000US17097,"Lake County, Illinois",355918,2280,4.4,0.2,17,097
4,0500000US17127,"Massac County, Illinois",5602,268,0.3,0.5,17,127


## Step 2: Clean Data
Data cleaning is an important step in the data science process. This step is often the hardest and most time consuming. 

### 2.1. Clean ACS Data
#### 2.1.1 Make sure variable types are set correctly
For the ACS data we have both population counts and precents. The percentage estiamtes are not integers - they are "floats" which refers the the decimal place that can float in the number between the integer and the fractional part. Here is more information on Python number types https://www.tutorialspoint.com/python/python_numbers.htm 

In [ ]:
import json 

for vintage in vintages:
  for variable in get_vars.split(","):
    variable_metadata_hyperlink = (f'https://api.census.gov/data/{vintage}/{dataset_name}/variables/{variable}.json')
    # Obtain Census API JSON Data
    !wget --no-cache --quiet --backups=1 {variable_metadata_hyperlink}

    with open(f"{variable}.json", "r") as rf:
      variable_metadata = json.load(rf)

    # Find the variable label 
    census_label_string = str(variable_metadata["label"])
    last_exclamation_point_position = census_label_string.rfind("!!")
    if last_exclamation_point_position >= 0:
      last_exclamation_point_position = last_exclamation_point_position + 2
    else:
      last_exclamation_point_position = 0
    label = census_label_string[last_exclamation_point_position:] 

    # Add vintage to label name (skip geo_id and name variables)
    if variable not in ['GEO_ID','NAME']:
      label_addvintage = label + f' {vintage}'
    else:
      label_addvintage = label

    # Add estimate or Margin of Error to label
    last_letter_of_variable = variable[-1]
    if variable not in ['GEO_ID','NAME']:
      if last_letter_of_variable == 'E':
        label_addvintage_addtype = label_addvintage + ' (Estimate)'
      if last_letter_of_variable == 'M':
        label_addvintage_addtype = label_addvintage + ' (MOE)'
    else:
      label_addvintage_addtype = label_addvintage
    print(vintage,"Renameing",variable," = ",label_addvintage_addtype,"Changing type to",variable_metadata["predicateType"])

    # Change variable type
    acs_df[vintage][variable] = acs_df[vintage][variable].astype(variable_metadata["predicateType"])

    acs_df[vintage] = acs_df[vintage].rename(columns={variable: label_addvintage_addtype}) 




2012 Renameing GEO_ID  =  Geography Changing type to string
2012 Renameing NAME  =  Geographic Area Name Changing type to string
2012 Renameing S0801_C01_001E  =  Workers 16 years and over 2012 (Estimate) Changing type to int
2012 Renameing S0801_C01_001M  =  Workers 16 years and over 2012 (MOE) Changing type to int
2012 Renameing S0801_C01_009E  =  Public transportation (excluding taxicab) 2012 (Estimate) Changing type to float
2012 Renameing S0801_C01_009M  =  Public transportation (excluding taxicab) 2012 (MOE) Changing type to float
2019 Renameing GEO_ID  =  Geography Changing type to string
2019 Renameing NAME  =  Geographic Area Name Changing type to string
2019 Renameing S0801_C01_001E  =  Workers 16 years and over 2019 (Estimate) Changing type to int
2019 Renameing S0801_C01_001M  =  Workers 16 years and over 2019 (MOE) Changing type to int
2019 Renameing S0801_C01_009E  =  Public transportation (excluding taxicab) 2019 (Estimate) Changing type to float
2019 Renameing S0801_C01

In [ ]:
acs_df['2012'].head()

,Geography,Geographic Area Name,Workers 16 years and over 2012 (Estimate),Workers 16 years and over 2012 (MOE),Public transportation (excluding taxicab) 2012 (Estimate),Public transportation (excluding taxicab) 2012 (MOE),state,county
0,0500000US29115,"Linn County, Missouri",5052,234,0.0,0.1,29,115
1,0500000US29091,"Howell County, Missouri",15623,560,0.3,0.3,29,091
2,0500000US29101,"Johnson County, Missouri",25227,677,0.3,0.2,29,101
3,0500000US29105,"Laclede County, Missouri",15019,481,0.1,0.2,29,105
4,0500000US29125,"Maries County, Missouri",3830,246,0.4,0.6,29,125


#### 2.1.2 Use descriptive statistics to check cleaning
A descriptive statistics table is a great way to check to make sure the variables have been created correctly.

In [ ]:
float_col_list = list(acs_df['2012'].select_dtypes(include=['float']).columns)
table1 = acs_df['2012'][float_col_list].describe().T
varformat = "{:,.2f}" # The variable format adds a comma and rounds up
table_title = "Table 1. Descriptive statistics for variables by county, 2012 5-year ACS."
table1 = table1.style.set_caption(table_title).format(varformat).set_properties(**{'text-align': 'right'})
table1

,count,mean,std,min,25%,50%,75%,max
Public transportation (excluding taxicab) 2012 (Estimate),"3,221.00",0.97,3.00,0.00,0.10,0.40,0.80,60.80
Public transportation (excluding taxicab) 2012 (MOE),"3,221.00",0.66,1.46,0.10,0.20,0.30,0.70,40.00


In [ ]:
float_col_list = list(acs_df['2019'].select_dtypes(include=['float']).columns)
table2 = acs_df['2019'][float_col_list].describe().T
varformat = "{:,.2f}" # The variable format adds a comma and rounds up
table_title = "Table 2. Descriptive statistics for variables by county, 2019 5-year ACS."
table2 = table2.style.set_caption(table_title).format(varformat).set_properties(**{'text-align': 'right'})
table2

,count,mean,std,min,25%,50%,75%,max
Public transportation (excluding taxicab) 2019 (Estimate),"3,220.00",0.91,3.05,0.00,0.10,0.30,0.70,61.20
Public transportation (excluding taxicab) 2019 (MOE),"3,220.00",0.67,1.55,0.10,0.20,0.30,0.60,59.00


## Step 3: Merge Data together
Merging data together is a powerful tool in data science workflows. Merge data refers to combining data that represent the same unit of analysis. In this example all three datasets represent counties in the US. Each dataframe has a common variable called GEO_ID which uniquely identifies the counties.

Here is a good overview of data merging in python:
https://towardsdatascience.com/why-and-how-to-use-merge-with-pandas-in-python-548600f7e738

### 3.1 - Merge Part 1 - Combine ACS Data

In [ ]:
acs_df['2012'].head()

,Geography,Geographic Area Name,Workers 16 years and over 2012 (Estimate),Workers 16 years and over 2012 (MOE),Public transportation (excluding taxicab) 2012 (Estimate),Public transportation (excluding taxicab) 2012 (MOE),state,county
0,0500000US29115,"Linn County, Missouri",5052,234,0.0,0.1,29,115
1,0500000US29091,"Howell County, Missouri",15623,560,0.3,0.3,29,091
2,0500000US29101,"Johnson County, Missouri",25227,677,0.3,0.2,29,101
3,0500000US29105,"Laclede County, Missouri",15019,481,0.1,0.2,29,105
4,0500000US29125,"Maries County, Missouri",3830,246,0.4,0.6,29,125


In [ ]:
acs_df['2019'].head()

,Geography,Geographic Area Name,Workers 16 years and over 2019 (Estimate),Workers 16 years and over 2019 (MOE),Public transportation (excluding taxicab) 2019 (Estimate),Public transportation (excluding taxicab) 2019 (MOE),state,county
0,0500000US17051,"Fayette County, Illinois",8633,353,0.8,0.4,17,051
1,0500000US17107,"Logan County, Illinois",12211,378,0.4,0.3,17,107
2,0500000US17165,"Saline County, Illinois",9631,334,0.9,0.6,17,165
3,0500000US17097,"Lake County, Illinois",355918,2280,4.4,0.2,17,097
4,0500000US17127,"Massac County, Illinois",5602,268,0.3,0.5,17,127


In [ ]:
merge_df = pd.merge(left = acs_df['2012'],
                    right = acs_df['2019'],
                    left_on = ['Geography','Geographic Area Name'],
                    right_on = ['Geography','Geographic Area Name'],
                    how = 'inner')
merge_df.head()

,Geography,Geographic Area Name,Workers 16 years and over 2012 (Estimate),Workers 16 years and over 2012 (MOE),Public transportation (excluding taxicab) 2012 (Estimate),Public transportation (excluding taxicab) 2012 (MOE),state_x,county_x,Workers 16 years and over 2019 (Estimate),Workers 16 years and over 2019 (MOE),Public transportation (excluding taxicab) 2019 (Estimate),Public transportation (excluding taxicab) 2019 (MOE),state_y,county_y
0,0500000US29115,"Linn County, Missouri",5052,234,0.0,0.1,29,115,5579,158,0.0,0.5,29,115
1,0500000US29091,"Howell County, Missouri",15623,560,0.3,0.3,29,091,16210,514,0.4,0.4,29,091
2,0500000US29101,"Johnson County, Missouri",25227,677,0.3,0.2,29,101,26742,682,0.2,0.1,29,101
3,0500000US29105,"Laclede County, Missouri",15019,481,0.1,0.2,29,105,15395,663,0.0,0.2,29,105
4,0500000US29125,"Maries County, Missouri",3830,246,0.4,0.6,29,125,3716,254,0.2,0.3,29,125


In [ ]:
float_col_list = list(merge_df.select_dtypes(include=['float']).columns)
table3 = merge_df[float_col_list].describe().T
varformat = "{:,.2f}" # The variable format adds a comma and rounds up
table_title = "Table 3. Descriptive statistics for variables by county, 2012 & 2019 5-year ACS."
table3 = table3.style.set_caption(table_title).format(varformat).set_properties(**{'text-align': 'right'})
table3

,count,mean,std,min,25%,50%,75%,max
Public transportation (excluding taxicab) 2012 (Estimate),"3,217.00",0.97,3.00,0.00,0.10,0.40,0.80,60.80
Public transportation (excluding taxicab) 2012 (MOE),"3,217.00",0.66,1.46,0.10,0.20,0.30,0.70,40.00
Public transportation (excluding taxicab) 2019 (Estimate),"3,217.00",0.91,3.05,0.00,0.10,0.30,0.70,61.20
Public transportation (excluding taxicab) 2019 (MOE),"3,217.00",0.67,1.55,0.10,0.20,0.30,0.60,59.00


### 3.3 Clean Up Merge File
Notice that the merge file has several columns (variables) that have repeated information. All three datafiles had a column called `name`. The name column is a good way to see that the merge has worked correctly. In this step we will also rename the columns.

### 3.3.1 Remove repeated columns

In [ ]:
## The state and county columns also match - we can drop the repeated columns
merge_df = merge_df.drop(['state_y','county_y'], axis = 1)
merge_df.head()

,Geography,Geographic Area Name,Workers 16 years and over 2012 (Estimate),Workers 16 years and over 2012 (MOE),Public transportation (excluding taxicab) 2012 (Estimate),Public transportation (excluding taxicab) 2012 (MOE),state_x,county_x,Workers 16 years and over 2019 (Estimate),Workers 16 years and over 2019 (MOE),Public transportation (excluding taxicab) 2019 (Estimate),Public transportation (excluding taxicab) 2019 (MOE)
0,0500000US29115,"Linn County, Missouri",5052,234,0.0,0.1,29,115,5579,158,0.0,0.5
1,0500000US29091,"Howell County, Missouri",15623,560,0.3,0.3,29,091,16210,514,0.4,0.4
2,0500000US29101,"Johnson County, Missouri",25227,677,0.3,0.2,29,101,26742,682,0.2,0.1
3,0500000US29105,"Laclede County, Missouri",15019,481,0.1,0.2,29,105,15395,663,0.0,0.2
4,0500000US29125,"Maries County, Missouri",3830,246,0.4,0.6,29,125,3716,254,0.2,0.3


### 3.3.2 Add new variable
Add difference between ACS statiscs

In [ ]:
percent1 = 'Public transportation (excluding taxicab) 2019 (Estimate)'
percent2 = 'Public transportation (excluding taxicab) 2012 (Estimate)'
merge_df.loc[:,'Difference between Percents'] = (merge_df[percent1]) - (merge_df[percent2])
merge_df['Difference between Percents'].describe()

count    3217.000000
mean       -0.067454
std         0.810308
min       -17.000000
25%        -0.300000
50%         0.000000
75%         0.200000
max        18.400000
Name: Difference between Percents, dtype: float64

Add standard error for the pooled estimate

In [ ]:
moe1 = 'Public transportation (excluding taxicab) 2019 (MOE)'
moe2 = 'Public transportation (excluding taxicab) 2012 (MOE)'
census_critical_value = 1.645
merge_df.loc[:,'Standard Error 1'] = (merge_df[moe1])/census_critical_value
merge_df.loc[:,'Standard Error 2'] = (merge_df[moe2])/census_critical_value
merge_df.loc[:,'Difference Standard Error'] = (merge_df['Standard Error 1']**2 + merge_df['Standard Error 2']**2)**(1/2)
merge_df['Difference Standard Error'].describe()

count    3217.000000
mean        3.725210
std         2.794229
min         0.023168
25%         1.661701
50%         3.054688
75%         5.201947
max        11.631907
Name: Difference Standard Error, dtype: float64

Add test statistic for difference

In [ ]:
merge_df.loc[:,'Test Statistic'] = abs(merge_df['Difference between Percents'] / merge_df['Difference Standard Error'])
merge_df['Test Statistic'].describe()

count    3217.000000
mean        0.419156
std         2.664195
min         0.000000
25%         0.017194
50%         0.060790
75%         0.203897
max        74.078825
Name: Test Statistic, dtype: float64

## Step 4: Explore Data
### 4.1 - Which Counties had significant changes?

In [ ]:
merge_df.loc[(merge_df['Test Statistic'] >= 1.645) & 
             (merge_df['Difference between Percents'] > 0) &
             (merge_df['state_x'] == '48')]

,Geography,Geographic Area Name,Workers 16 years and over 2012 (Estimate),Workers 16 years and over 2012 (MOE),Public transportation (excluding taxicab) 2012 (Estimate),Public transportation (excluding taxicab) 2012 (MOE),state_x,county_x,Workers 16 years and over 2019 (Estimate),Workers 16 years and over 2019 (MOE),Public transportation (excluding taxicab) 2019 (Estimate),Public transportation (excluding taxicab) 2019 (MOE),Difference between Percent,Difference between Percents,Standard Error 1,Standard Error 2,Difference Standard Error,Test Statistic
603,0500000US48153,"Floyd County, Texas",2841,179,0.0,1.3,48,153,2415,158,1.4,1.6,1.4,1.4,0.972644,0.790274,0.797942,1.754513
3057,0500000US48269,"King County, Texas",147,47,0.0,22.3,48,269,113,32,5.3,5.6,5.3,5.3,3.404255,13.556231,0.071545,74.078825
3154,0500000US48103,"Crane County, Texas",1664,126,0.0,2.2,48,103,2100,167,1.9,1.6,1.9,1.9,0.972644,1.337386,0.604714,3.141981


In [ ]:
merge_df.loc[(merge_df['Workers 16 years and over 2012 (Estimate)'] > 50000) &
             (merge_df['Difference between Percents'] > 0) &
             (merge_df['state_x'] == '48')]

,Geography,Geographic Area Name,Workers 16 years and over 2012 (Estimate),Workers 16 years and over 2012 (MOE),Public transportation (excluding taxicab) 2012 (Estimate),Public transportation (excluding taxicab) 2012 (MOE),state_x,county_x,Workers 16 years and over 2019 (Estimate),Workers 16 years and over 2019 (MOE),Public transportation (excluding taxicab) 2019 (Estimate),Public transportation (excluding taxicab) 2019 (MOE),Difference between Percent,Difference between Percents,Standard Error 1,Standard Error 2,Difference Standard Error,Test Statistic
587,0500000US48485,"Wichita County, Texas",61365,1185,0.4,0.2,48,485,60792,1204,0.6,0.2,0.2,0.2,0.121581,0.121581,5.815953,0.034388
606,0500000US48027,"Bell County, Texas",141453,1753,0.3,0.1,48,027,156345,2006,0.5,0.2,0.2,0.2,0.121581,0.060790,7.356664,0.027186
616,0500000US48215,"Hidalgo County, Texas",274317,3048,0.3,0.1,48,215,321731,3145,0.4,0.1,0.1,0.1,0.060790,0.060790,11.631907,0.008597
847,0500000US48491,"Williamson County, Texas",208257,1832,0.5,0.1,48,491,275549,1905,0.9,0.1,0.4,0.4,0.060790,0.060790,11.631907,0.034388
3030,0500000US48121,"Denton County, Texas",342383,2506,0.7,0.1,48,121,445453,2252,0.8,0.1,0.1,0.1,0.060790,0.060790,11.631907,0.008597
3147,0500000US48091,"Comal County, Texas",50136,1095,0.1,0.1,48,091,65484,1117,0.2,0.1,0.1,0.1,0.060790,0.060790,11.631907,0.008597
